# How the algorithm works
### label real data using a service like roboflow/label real data usingg an old model
### put the labeled data (masks, images) in the folder temp
### run this line to get the polygons from the masks 


In [24]:
%run masks_to_polygons.py

### By this stage you should have three folders inside temp (images, labels, masks)
### run the following function (split_img_label), this function will take the temp data and transform it into a data set of images and lables each having their train and validation data set

In [1]:
from tqdm import tqdm
import subprocess

In [2]:
def split_img_label(data_train,data_test,folder_train,folder_test):
    
    # os.mkdir(folder_train)
    # os.mkdir(folder_test)
    main_dir = "dataset"
    sub_dirs = ["images", "labels"]
    train_val_dirs = ["train", "val"]
    
    # Create the main dataset directory
    os.makedirs(main_dir, exist_ok=True)
    
    # Create subdirectories and train/val folders
    for sub_dir in sub_dirs:
        # Create the folder for images and labels
        sub_dir_path = os.path.join(main_dir, sub_dir)
        os.makedirs(sub_dir_path, exist_ok=True)
        
        for train_val in train_val_dirs:
            # Create the train and val folders inside images and labels
            os.makedirs(os.path.join(sub_dir_path, train_val), exist_ok=True)

    
    train_ind=list(data_train.index)
    test_ind=list(data_test.index)
    folder_train_images="dataset\\images\\train"
    folder_train_labels="dataset\\labels\\train"
    folder_test_images="dataset\\images\\val"
    folder_test_labels="dataset\\labels\\val"
  
    # Train folder
    for i in tqdm(range(len(train_ind))):
        source ='.\\'+ "tmp\\images" + '\\'  +data_train[train_ind[i]].split('/')[3]
        destination = '.\\'+ folder_train_images + '\\'  +data_train[train_ind[i]].split('/')[3]
        command = f'copy "{source}" "{destination}"'
        os.system(command)
        source ='.\\'+ "tmp\\labels" + '\\'  +data_train[train_ind[i]].split('/')[3].split('.jpg')[0]+'.txt'
        destination = '.\\'+ folder_train_labels + '\\'  +data_train[train_ind[i]].split('/')[3].split('.jpg')[0]+'.txt'
        command = f'copy "{source}" "{destination}"'
        os.system(command)
     
# Execute the command
#     Test folder
    for j in tqdm(range(len(test_ind))):
        source ='.\\'+ "tmp\\images" + '\\'  +data_test[test_ind[j]].split('/')[3]
        destination = '.\\'+ folder_test_images + '\\'  +data_test[test_ind[j]].split('/')[3]
        command = f'copy "{source}" "{destination}"'
        os.system(command)
        source ='.\\'+ "tmp\\labels" + '\\'  +data_test[test_ind[j]].split('/')[3].split('.jpg')[0]+'.txt'
        destination = '.\\'+ folder_test_labels + '\\'  +data_test[test_ind[j]].split('/')[3].split('.jpg')[0]+'.txt'
        command = f'copy "{source}" "{destination}"'
        os.system(command)


### the  following code is only responsible for creating a list of the names of the image and label files into an array and running the split function on that indexing. !Be careful of the image type as it requires a minor change to jpg if such files exists 

In [3]:
import pandas as pd 
import os 
import sklearn.model_selection
from sklearn.model_selection import train_test_split
import glob

PATH_Images = './tmp/images/'
PATH_Labels= './tmp/labels/'

list_img=[img for img in os.listdir(PATH_Images) if img.endswith('.jpg')==True]# you can change this extension to jpg
list_txt=[img for img in os.listdir(PATH_Labels) if img.endswith('.txt')==True]

path_img=[]

for i in range (len(list_img)):
    path_img.append(PATH_Images+list_img[i])
    
df=pd.DataFrame(path_img)

# split 
data_train, data_test, labels_train, labels_test = train_test_split(df[0], df.index, test_size=0.20, random_state=42)
folder_train_name="train"
folder_test_name="test"
# Function split 
split_img_label(data_train,data_test,folder_train_name,folder_test_name)


100%|██████████████████████████████████████████████████████████████████████████████████| 66/66 [00:01<00:00, 40.59it/s]


### by this stage you should hava a ready to train data set

In [4]:
from ultralytics import YOLO

model = YOLO('runs/segment/train20/weights/best.pt')  # в случае добучения - указать путь до предобученной модели. базовая скачается автоматически
model.train(data='yolo.yaml', epochs=200, device=[0], imgsz=640, augment=True, lr0=0.0001,mask_ratio=2, batch=32)

New https://pypi.org/project/ultralytics/8.3.75 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.69  Python-3.9.21 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
engine\trainer: task=segment, mode=train, model=runs/segment/train20/weights/best.pt, data=yolo.yaml, epochs=200, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train32, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=2, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None,

train: Scanning D:\mask-to-annotation\datasets\dataset\labels\train... 261 images, 0 backgrounds, 0 corrupt: 100%|█████

train: New cache created: D:\mask-to-annotation\datasets\dataset\labels\train.cache



val: Scanning D:\mask-to-annotation\datasets\dataset\labels\val... 66 images, 0 backgrounds, 0 corrupt: 100%|██████████


val: New cache created: D:\mask-to-annotation\datasets\dataset\labels\val.cache
Plotting labels to runs\segment\train32\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.0001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\segment\train32
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP

WARNING  Model does not support 'augment=True', reverting to single-scale prediction.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP


                   all         66         66      0.999          1      0.995      0.931      0.999          1      0.995      0.893
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\segment\train32


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001A747937280>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.0410

In [3]:
torch.cuda.device_count()

1

In [6]:
torch.cuda.is_available(): True
torch.cuda.device_count(): 1
os.environ['CUDA_VISIBLE_DEVICES']: 1,2,3

SyntaxError: illegal target for annotation (442746295.py, line 1)

### by this stage you should have a model somewhere in the folder (runs/segment/train%d)
### move the images that you want to annotate to the folder "predicted_masks/images"
### Folder Structure Explanation

To run this notebook successfully, ensure that there is a folder named `predicted_masks` in the same directory as the notebook. Inside the `predicted_masks` folder, there should be three subfolders:

1. **images**: This folder contains the images for predictions.
2. **masks**: This folder contains the mask predictions corresponding to the images.
3. **overlay**: This folder contains overlay images (a combination of the original images and their predicted masks).

The structure should look li

    ./predicted_masks 
                    ├── images (put your images here)
                    ├── masks
                    └── overlay
                 this:



### the following code is only responsible for overlaying the mask on top of the image, it has two versions one for png images and the other for jpg, but be aware that you have to call the right version depending on what you need in the function "*save_prediction_masks"

In [5]:
import torch
torch.cuda.empty_cache()

In [6]:
from PIL import Image
import numpy as np
import cv2

def overlay_mask(image_path, mask, alpha=0.8):
    # Load the image
    # image = Image.open(image_path).convert("RGB")
    image = Image.open(image_path).convert("RGBA")
    # resized_image = image.resize(384,480)

    # Load the mask and convert it to an array
    mask = Image.open(mask_path).convert("L")  # Convert to grayscale
    mask = np.array(mask)

    # Create a colored version of the mask
    colored_mask = np.zeros((*mask.shape, 4), dtype=np.uint8)  # RGBA
    colored_mask[:, :, 0] = 255  # Red channel (color of the mask)
    colored_mask[:, :, 3] = mask * 255  # Alpha channel based on the mask

    # Convert the colored mask to an image
    colored_mask_image = Image.fromarray(colored_mask, 'RGBA')

    # Overlay the mask on the image with specified transparency
    overlay = Image.alpha_composite(image, colored_mask_image)

    # Save the result to the specified path
    overlay_path = "./predicted_masks/overlay/" + image_path.split('/')[3]
    overlay.save(overlay_path)
    
    # Convert overlay to format compatible with OpenCV (BGR) for further processing (if necessary)
    overlay_cv = cv2.cvtColor(np.array(overlay), cv2.COLOR_RGBA2BGR)
    cv2.imwrite(overlay_path, overlay_cv)

# Example usage:
# overlay_mask('./predicted_masks/images/00435-4194690285.png', './predicted_masks/masks/00435-4194690285.png')

In [8]:
from PIL import Image, ImageDraw
import cv2
import numpy as np
from shapely.geometry import Polygon, MultiPolygon

def overlay_mask_jpg(image_path, masks, alpha=0.1):
    """
    Overlays a YOLO segmentation mask onto an image, ensuring only one connected polygon is drawn.
    
    Args:
        image_path (str): Path to the input image.
        masks (list): List of YOLO masks (polygon coordinates).
        alpha (float): Transparency level for the overlay.
    
    Saves:
        Processed image with overlay mask.
    """
    overlay_path = "./predicted_masks/overlay/" + image_path.split('/')[-1]  # Use last part of path
    
    # Open image in PIL
    img_pil = Image.open(image_path).convert("RGBA")
    overlay = Image.new('RGBA', img_pil.size, (255, 255, 255, 0))
    overlay_draw = ImageDraw.Draw(overlay)

    all_polygons = []

    # Collect all polygons from masks
    for mask in masks:
        polygon = mask.xy[0]  # Get polygon coordinates
        if len(polygon) >= 3:
            all_polygons.append(Polygon(polygon))  # Convert to Shapely polygon

    if not all_polygons:
        print("No valid polygons detected.")
        return  # Exit if no valid polygons

    # Merge polygons and keep only the largest one
    merged = MultiPolygon(all_polygons).buffer(0)  # Merge overlapping polygons
    if isinstance(merged, MultiPolygon):
        largest_polygon = max(merged.geoms, key=lambda p: p.area)  # Keep the largest polygon
    else:
        largest_polygon = merged

    # Convert the largest polygon back to a format usable by ImageDraw
    polygon_coords = list(largest_polygon.exterior.coords)

    # Draw the largest polygon
    overlay_draw.polygon(polygon_coords, outline=(0, 255, 0), fill=(0, 255, 0, 127))

    # Draw a centroid marker
    centroid = largest_polygon.centroid
    circle_radius = 5
    left_up_point = (centroid.x - circle_radius, centroid.y - circle_radius)
    right_down_point = (centroid.x + circle_radius, centroid.y + circle_radius)
    overlay_draw.ellipse([left_up_point, right_down_point], fill=(255, 0, 0))

    # Merge overlay with original image
    img_pil = Image.alpha_composite(img_pil, overlay)

    # Convert PIL image to OpenCV format and save
    frame = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
    cv2.imwrite(overlay_path, frame)

    print(f"Overlay saved: {overlay_path}")


In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from tqdm import tqdm
def process_yolo_masks(masks):
    """
    Takes YOLO segmentation masks and returns a single connected binary mask.
    
    Args:
        masks (tensor): YOLOv8 segmentation mask tensor.
        
    Returns:
        final_mask (numpy.ndarray): Processed binary mask with only one connected component.
    """
    if masks is None or len(masks) == 0:
        return None  # No mask detected

    # Convert YOLO mask tensor to NumPy
    masks = masks.data.cpu().numpy()  # Shape: (num_masks, H, W)

    # Merge all masks into a single binary mask
    merged_mask = np.max(masks, axis=0)  # Combine multiple masks into one

    # Convert to uint8 format for OpenCV
    merged_mask = (merged_mask * 255).astype(np.uint8)

    # Apply morphological closing to fill small gaps
    kernel = np.ones((5, 5), np.uint8)
    merged_mask = cv2.morphologyEx(merged_mask, cv2.MORPH_CLOSE, kernel)

    # Find the largest connected component
    contours, _ = cv2.findContours(merged_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        final_mask = np.zeros_like(merged_mask)
        cv2.drawContours(final_mask, [max(contours, key=cv2.contourArea)], -1, 255, thickness=cv2.FILLED)
    else:
        final_mask = merged_mask  # If no contours found, keep original

    return final_mask


In [10]:
import os
from pathlib import Path
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import cv2
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO
# from pixel_size.utils import get_pixel_real_size
from weld_processing.read_mask import plot_mask_and_point,return_points_and_size
from PIL import Image, ImageDraw
from shapely.geometry import Polygon

def save_prediction_masks():
    PATH_Images="./predicted_masks/images"
    PATH_Masks="./predicted_masks/masks"
    
    list_img=[img for img in os.listdir(PATH_Images) if img.endswith('.jpg')==True]
    model_path = Path('runs/segment/train32/weights/best.pt')  # Adjust path as needed
    model = YOLO(model_path)
    for i in tqdm(list_img):
        img_path=PATH_Images+"/"+ i
        # Load the model (replace 'best.pt' with your actual model file name)
        print(img_path)
        # Load an image
        img = cv2.imread(img_path)

            # Resize the image
        # print(img)
        H, W,_ = img.shape
        results = model(img,imgsz=640, max_det=1, iou=0.7)
        
        # mask= results[0].masks.data[0].cpu().numpy() * 255
        mask= process_yolo_masks(results[0].masks)
        # print(mask.shape[0]/mask.shape[1])
        # mask = cv2.resize(mask, (W, H))
        # print(H/W)
    
        mask_path=PATH_Masks+'/'+ i
 
        cv2.imwrite(mask_path, mask)
        # print(mask_path, img_path)
        overlay_mask_jpg(img_path,results[0].masks[0])

      

# # res = get_pixel_real_size(img_path)
# # print(res)
# # top_line_len,top_coords, bot_line_len,bot_coords = return_points_and_size('./output.png')
# # print(f'top_line_len={top_line_len,top_line_len*res[0]}mm, bot_line_len={bot_line_len,bot_line_len*res[0]}')

# # plot_mask_and_point('./output.png')
save_prediction_masks()


  0%|                                                                                          | 0/136 [00:00<?, ?it/s]

./predicted_masks/images/10a.jpg

0: 480x640 1 weld, 5.5ms
Speed: 2.1ms preprocess, 5.5ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)


  1%|▌                                                                                 | 1/136 [00:00<00:23,  5.84it/s]

Overlay saved: ./predicted_masks/overlay/10a.jpg
./predicted_masks/images/10b.jpg

0: 480x640 1 weld, 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)
Overlay saved: ./predicted_masks/overlay/10b.jpg
./predicted_masks/images/10c.jpg

0: 480x640 1 weld, 6.5ms
Speed: 2.5ms preprocess, 6.5ms inference, 3.5ms postprocess per image at shape (1, 3, 480, 640)
Overlay saved: ./predicted_masks/overlay/10c.jpg

  2%|█▊                                                                                | 3/136 [00:00<00:15,  8.81it/s]

...

Overlay saved: ./predicted_masks/overlay/9b.jpg
./predicted_masks/images/9c.jpg

0: 480x640 1 weld, 22.8ms
Speed: 2.5ms preprocess, 22.8ms inference, 5.4ms postprocess per image at shape (1, 3, 480, 640)


 99%|███████████████████████████████████████████████████████████████████████████████▍| 135/136 [00:14<00:00,  8.26it/s]

Overlay saved: ./predicted_masks/overlay/9c.jpg
./predicted_masks/images/9d.jpg

0: 480x640 1 weld, 32.4ms
Speed: 3.2ms preprocess, 32.4ms inference, 4.7ms postprocess per image at shape (1, 3, 480, 640)


100%|████████████████████████████████████████████████████████████████████████████████| 136/136 [00:15<00:00,  9.02it/s]

Overlay saved: ./predicted_masks/overlay/9d.jpg


#### by this stage you should have your overlayed images and masks in the folder "predicted_masks"
#### you can then choose to add those newly annotated data to your data set and train a new iteration

In [21]:
import shutil
def add_to_dataset():
    path_images = './tmp/images/'
    path_masks= './tmp/masks/'
    path_masks_predicted='./predicted_masks/masks/'
    path_images_predicted='./predicted_masks/images/'
    path_overlay='./predicted_masks/overlay/'
    list_img=[img for img in os.listdir(path_overlay) if img.endswith('.jpg')==True]
    for i in tqdm(list_img):
        img_path_predicted=path_images_predicted+"/"+ i
        mask_path_predicted=path_masks_predicted+"/"+ i
          # Move predicted image to the image directory
        if os.path.exists(img_path_predicted):
            shutil.move(img_path_predicted, path_images)
        
        # Move predicted mask to the masks directory
        if os.path.exists(mask_path_predicted):
            shutil.move(mask_path_predicted, path_masks)
        
add_to_dataset()  

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: './predicted_masks/overlay/'

In [59]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Load the model
model = YOLO('runs/segment/train25/weights/best.pt')

# Load the image
img_path = "./predicted_masks/images/6a.jpg"
img = cv2.imread(img_path)

# Run inference
results = model(img)

# Get segmentation mask
masks =process_yolo_masks(results[0].masks)
cv2.imshow("Processed Single Mask", masks)
cv2.waitKey(0)  # Wait for a key press
cv2.destroyAllWindows()  # Close the window


0: 480x640 1 weld, 17.6ms
Speed: 2.0ms preprocess, 17.6ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)


# Show the loss functions in more detail

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob

# Find the latest training run folder inside "runs/segment/"
runs_path = "runs/segment/"
train_folders = sorted(glob.glob(os.path.join(runs_path, "train*")), key=os.path.getmtime, reverse=True)

if not train_folders:
    raise FileNotFoundError("No training folders found in 'runs/segment/'")

latest_run_path = train_folders[0]  # Get the most recent training folder
results_csv_path = os.path.join(latest_run_path, "results.csv")

# Load the training log
df = pd.read_csv(results_csv_path)

# Plot segmentation loss
plt.figure(figsize=(10, 5))
plt.plot(df["epoch"], df["train/seg_loss"], label="Train Segmentation Loss")
plt.plot(df["epoch"], df["val/seg_loss"], label="Validation Segmentation Loss", linestyle="--")

plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("YOLO Segmentation Loss Curve")
plt.legend()
plt.grid()
# plt.show()
# Save the plot inside the latest run folder
plot_path = os.path.join(latest_run_path, "segmentation_loss.png")
plt.savefig(plot_path, dpi=300)
plt.close()

print(f"Segmentation loss plot saved at: {plot_path}")


Segmentation loss plot saved at: runs/segment\train32\segmentation_loss.png


In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob

# Find the latest training run folder inside "runs/segment/"
runs_path = "runs/segment/"
train_folders = sorted(glob.glob(os.path.join(runs_path, "train*")), key=os.path.getmtime, reverse=True)

if not train_folders:
    raise FileNotFoundError("No training folders found in 'runs/segment/'")

latest_run_path = train_folders[0]  # Get the most recent training folder
results_csv_path = os.path.join(latest_run_path, "results.csv")

# Load the training log
df = pd.read_csv(results_csv_path)

# Plot segmentation loss
plt.figure(figsize=(10, 5))
plt.plot(df["epoch"], df["train/box_loss"], label="Train box Loss")
plt.plot(df["epoch"], df["val/box_loss"], label="Validation box Loss", linestyle="--")

plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("YOLO Box Loss Curve")
plt.legend()
plt.grid()
# plt.show()
# Save the plot inside the latest run folder
plot_path = os.path.join(latest_run_path, "box_loss.png")
plt.savefig(plot_path, dpi=300)
plt.close()

print(f"Segmentation loss plot saved at: {plot_path}")


Segmentation loss plot saved at: runs/segment\train32\box_loss.png


# smooth the masks 

In [32]:
import cv2
import numpy as np
import glob
import os

# Set paths
input_mask_folder = "./predicted_masks/masks"  # Change this to your YOLO mask folder
output_mask_folder = "./predicted_masks/smoothed_masks"

# Create output directory if not exists
os.makedirs(output_mask_folder, exist_ok=True)

# Get all mask files
mask_files = glob.glob(os.path.join(input_mask_folder, "*.jpg"))  # Change extension if needed

# Process each mask
for mask_path in mask_files:
    # Load mask as grayscale
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply morphological closing to fill small holes
    kernel = np.ones((5, 5), np.uint8)  # Adjust kernel size for more/less smoothing
    mask_closed = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    # Apply Gaussian Blur to soften edges
    mask_blurred = cv2.GaussianBlur(mask_closed, (5, 5), 0)
    
    # Threshold back to binary
    _, smoothed_mask = cv2.threshold(mask_blurred, 127, 255, cv2.THRESH_BINARY)

    # Save the smoothed mask
    output_path = os.path.join(output_mask_folder, os.path.basename(mask_path))
    cv2.imwrite(output_path, smoothed_mask)
contours, _ = cv2.findContours(smoothed_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for cnt in contours:
    epsilon = 0.01 * cv2.arcLength(cnt, True)  # Adjust smoothing factor
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    cv2.drawContours(smoothed_mask, [approx], -1, (255, 255, 255), thickness=cv2.FILLED)

print(f"Smoothed masks saved in: {output_mask_folder}")


Smoothed masks saved in: ./predicted_masks/smoothed_masks


In [67]:
import cv2
import numpy as np
import glob
import os
from scipy.interpolate import splprep, splev

# Set paths
input_mask_folder = "./predicted_masks/masks"  # Change this to your YOLO mask folder
output_mask_folder = "./predicted_masks/smoothed_masks"

# Create output directory if not exists
os.makedirs(output_mask_folder, exist_ok=True)

# Approximation Factor (Lower = Smoother)
EPSILON_FACTOR = 0.0015  # Reduce for more points, increase for simpler shapes

# Process each mask
for mask_path in glob.glob(os.path.join(input_mask_folder, "*.jpg")):
    # Load mask as grayscale
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a blank mask for the new smoothed contours
    smooth_mask = np.zeros_like(mask)

    # Process each contour
    for cnt in contours:
        if len(cnt) >= 3:  # Ensure it has enough points
            epsilon = EPSILON_FACTOR * cv2.arcLength(cnt, True)  # Compute epsilon
            smoothed_cnt = cv2.approxPolyDP(cnt, epsilon, True)  # Approximate contour
            cv2.drawContours(smooth_mask, [smoothed_cnt], -1, 255, thickness=cv2.FILLED)

    # Save the smoothed mask
    output_path = os.path.join(output_mask_folder, os.path.basename(mask_path))
    cv2.imwrite(output_path, smooth_mask)

print(f"Smoothed masks saved in: {output_mask_folder}")


Smoothed masks saved in: ./predicted_masks/smoothed_masks
